In [0]:
! pip install surprise

     |████████████████████████████████| 6.5MB 3.1MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1678246 sha256=ca21914f2572e763428320bee38f4c3416f6be9561ace774066a87a9a35f7587
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [0]:
import os
path = "/content/drive/My Drive/data"
os.chdir(path)
os.listdir(path)

['movies.csv', 'ratings.csv']

In [0]:
!pip install surprise

     |████████████████████████████████| 6.5MB 1.4MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1678232 sha256=b7f0a05cf4ec3ac0d2c6ddf11a9cd0a6a3b0b5291d5b064c8cb517638c23402f
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [13]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# File  : movie_surprise.py
# Author: WangYu
# Date  : 2020-03-07

from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise import BaselineOnly
from surprise.model_selection import KFold
from surprise.model_selection import GridSearchCV
import pandas as pd
# 数据读取
# sep分隔符，line_format行的形式 skip_lines 跳过的行数

def data_process():
    # 建立名字序号的dict,以及查看总共电影类别
    data = pd.read_csv('movies.csv', encoding = 'unicode_escape')
    rid_to_name = {}
    name_to_rid = {}
    category = set()
    for i in range(len(data['movieId'])):
        rid_to_name[data['movieId'][i]] = data['title'][i]
        name_to_rid[data['title'][i]] = data['movieId'][i]
        cat = data['genres'][i].split('|')
        for temp in cat:
            category.add(temp)
    return rid_to_name, name_to_rid,category

def main():
    rid_to_name, name_to_rid, category = data_process()
    size = len(category)
    print(size)     #20类
    # 数据读取
    reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)
    data = Dataset.load_from_file('./ratings.csv', reader=reader)
    train_set = data.build_full_trainset()

    #SVD biasSVD矩阵分解
    para_grid = {'n_epochs': [5],
                 'lr_all': [0.002],
                 'reg_all': [0.4]}
    gs = GridSearchCV(SVD, para_grid, measures=['rmse'], cv=3)
    print('biasSVD begin')
    gs.fit(data)
    print(gs.best_score['rmse'])
    print(gs.best_params['rmse'])

    model = gs.best_estimator['rmse']
    model.fit(train_set)
    uid = str(196)
    iid = str(302)
    # 输出uid对iid的预测结果
    pred = model.predict(uid, iid, r_ui=4, verbose=True)
    print(pred)

if __name__ == '__main__':
    main()




20
SVD begin
0.9052438960859729
{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.4}
user: 196        item: 302        r_ui = 4.00   est = 3.85   {'was_impossible': False}
user: 196        item: 302        r_ui = 4.00   est = 3.85   {'was_impossible': False}


In [20]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# File  : movie_surprise.py
# Author: WangYu
# Date  : 2020-03-07

from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise import BaselineOnly
from surprise.model_selection import KFold
from surprise.model_selection import GridSearchCV
import pandas as pd
# 数据读取
# sep分隔符，line_format行的形式 skip_lines 跳过的行数

def data_process():
    # 建立名字序号的dict,以及查看总共电影类别
    data = pd.read_csv('movies.csv', encoding = 'unicode_escape')
    rid_to_name = {}
    name_to_rid = {}
    category = set()
    for i in range(len(data['movieId'])):
        rid_to_name[data['movieId'][i]] = data['title'][i]
        name_to_rid[data['title'][i]] = data['movieId'][i]
        cat = data['genres'][i].split('|')
        for temp in cat:
            category.add(temp)
    return rid_to_name, name_to_rid,category

def main():
    rid_to_name, name_to_rid, category = data_process()
    size = len(category)
    print(size)     #20类
    # 数据读取
    reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)
    data = Dataset.load_from_file('./ratings.csv', reader=reader)
    train_set = data.build_full_trainset()

    #SVD funkSVD矩阵分解
    para_grid = {'n_epochs': [5],
                 'lr_all': [0.002],
                 'reg_all': [0.4],
                 'biased' : [False]}
    gs = GridSearchCV(SVD, para_grid, measures=['rmse'], cv=3)
    print('funkSVD begin')
    gs.fit(data)
    print(gs.best_score['rmse'])
    print(gs.best_params['rmse'])

    model = gs.best_estimator['rmse']
    model.fit(train_set)
    uid = str(196)
    iid = str(302)
    # 输出uid对iid的预测结果
    pred = model.predict(uid, iid, r_ui=4, verbose=True)
    print(pred)

if __name__ == '__main__':
    main()

20
funkSVD begin
1.4263052208556999
{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.4, 'biased': False}
user: 196        item: 302        r_ui = 4.00   est = 2.60   {'was_impossible': False}
user: 196        item: 302        r_ui = 4.00   est = 2.60   {'was_impossible': False}


In [28]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# File  : movie_surprise.py
# Author: WangYu
# Date  : 2020-03-07

from surprise import SVDpp
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise import BaselineOnly
from surprise.model_selection import KFold
from surprise.model_selection import GridSearchCV
import pandas as pd
# 数据读取
# sep分隔符，line_format行的形式 skip_lines 跳过的行数

def data_process():
    # 建立名字序号的dict,以及查看总共电影类别
    data = pd.read_csv('movies.csv', encoding = 'unicode_escape')
    rid_to_name = {}
    name_to_rid = {}
    category = set()
    for i in range(len(data['movieId'])):
        rid_to_name[data['movieId'][i]] = data['title'][i]
        name_to_rid[data['title'][i]] = data['movieId'][i]
        cat = data['genres'][i].split('|')
        for temp in cat:
            category.add(temp)
    return rid_to_name, name_to_rid,category

def main():
    rid_to_name, name_to_rid, category = data_process()
    size = len(category)
    print(size)     #20类
    # 数据读取
    reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)
    data = Dataset.load_from_file('./ratings.csv', reader=reader)
    train_set = data.build_full_trainset()

    #SVD SVDpp矩阵分解
    para_grid = {'n_epochs': [2],
                 'verbose' : [True]}
    gs = GridSearchCV(SVDpp, para_grid, measures=['rmse'], cv=2)
    print('SVD++ begin')
    gs.fit(data)
    print(gs.best_score['rmse'])
    print(gs.best_params['rmse'])

    model = gs.best_estimator['rmse']
    model.fit(train_set)
    uid = str(196)
    iid = str(302)
    # 输出uid对iid的预测结果
    pred = model.predict(uid, iid, r_ui=4, verbose=True)
    print(pred)

if __name__ == '__main__':
    main()


20
SVD++ begin
 processing epoch 0
 processing epoch 1
 processing epoch 0
 processing epoch 1
0.8906989312642716
{'n_epochs': 2, 'verbose': True}
 processing epoch 0
 processing epoch 1
user: 196        item: 302        r_ui = 4.00   est = 3.89   {'was_impossible': False}
user: 196        item: 302        r_ui = 4.00   est = 3.89   {'was_impossible': False}
